In [1]:
# https://invest.readthedocs.io/en/3.3.3/api/natcap.invest.seasonal_water_yield.html
# https://invest.readthedocs.io/en/3.3.3/scripting.html

In [4]:
import os 
from osgeo import gdal
gdal.UseExceptions()
gdal.PushErrorHandler('CPLQuietErrorHandler')
from natcap.invest.seasonal_water_yield import seasonal_water_yield
from itertools import product


In [5]:
alphas = [0.0833] # 0.0833, 
betas = [1] #[0.2, 0.6, 1]
gammas = [1] #[0.5, 1]
tfas = [500]
flows_dir = ["MFD"]

In [6]:
anos = list(range(2008, 2025))

In [7]:
def montar_args(ano, tfa, alpha, beta, gamma, flow_dir):

    #factor = 1.2
    folder_name = f'../output_data/final/{ano}/tfa{tfa}_alpha{alpha}_beta{beta}_gamma{gamma}_flow{flow_dir}' #_kc_adjusted_{factor}
    results_suffix = f"_{ano}_tfa{tfa}_alpha{alpha}_beta{beta}_gamma{gamma}_flow{flow_dir}" #_kc_adjusted
    biophysical_table_path = f'./01_lulc/biophysical_data/biophysical_table_30m_{ano}.csv' #aet_adjust_{factor}/
    
    
    os.makedirs(folder_name, exist_ok=True)

    args = {
        'workspace_dir': folder_name,
        'results_suffix': results_suffix,
        'threshold_flow_accumulation': tfa,
        'et0_dir': f'./03_et0/ET0_monthly/{ano}',
        'precip_dir': f'./04_precipitation/chirps_v3_latam/Precipitation_monthly/{ano}',
        'dem_raster_path': './02_dem/cropped_resampled_dem.tif',
        'lulc_raster_path': f'./01_lulc/lulc_data/resampled_land_use_30m_{ano}.tif',
        'soil_group_path': './05_soil_hydrologic_group/soil_group.tif',
        'aoi_path': 'bh_lago_paranoa_31983.geojson',
        'biophysical_table_path': biophysical_table_path, 
        'alpha_m': alpha,
        'beta_i': beta,
        'gamma': gamma,
        'user_defined_local_recharge': False,
        'user_defined_climate_zones': True,
        'climate_zone_table_path': f'./06_climate_zone/dados_pluviometricos/{ano}/climate_zone.csv',
        'climate_zone_raster_path': f'./06_climate_zone/dados_pluviometricos/{ano}/climate_zone.tif',
        'monthly_alpha': False,
        "flow_dir_algorithm": flow_dir
    }

    return args


In [8]:
## if __name__ == '__main__':
for ano in anos:

    for alpha, beta, gamma, tfa, flow_dir in product(alphas, betas, gammas, tfas, flows_dir):
        args = montar_args(ano, tfa, alpha, beta, gamma, flow_dir)
        print(f'Ano: {ano} | ALPHA: {alpha} | BETA: {beta} | GAMMA: {gamma} | FLOW DIR: {flow_dir}')
        seasonal_water_yield.execute(args)




Ano: 2008 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2009 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2010 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2011 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2012 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2013 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2014 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2015 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2016 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2017 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2018 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2019 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2020 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2021 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2022 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW DIR: MFD
Ano: 2023 | ALPHA: 0.0833 | BETA: 1 | GAMMA: 1 | FLOW D

In [ ]:
## testando o warning ##

# Warning 1: Starting with GDAL 3.7, PIXELTYPE=SIGNEDBYTE is no longer used to signal signed 8-bit raster. 
# Change your code to test for the new GDT_Int8 data type instead.
# Signed bytes can only store values from -128 to 127 so values >= 128 will overflow and corrupt your data.


In [ ]:
import rasterio
import numpy as np

# Lista de rasters a verificar
raster_paths = [
    args['dem_raster_path'],
    args['lulc_raster_path'],
    args['soil_group_path'],
    args['climate_zone_raster_path'],
    "./04_precipitation/chirps_v3_latam/1986/resampled_cropped_chirps-v3.0.1986.12.tif",
    f'./03_et0/ET0_monthly/1986/et0_1986_2.tif'
]

for path in raster_paths:
    try:
        with rasterio.open(path) as src:
            dtype = src.dtypes[0]
            if dtype == 'int8':
                print(f"⚠️ AVISO: O raster {path} usa 8 bits com sinal (int8).")
                print(f"   - Valores podem estar entre -128 e 127.")
                
                # Verifica se há valores problemáticos
                data = src.read(1)
                if (data > 127).any():
                    print(f"   🚨 VALORES PROBLEMÁTICOS: {np.unique(data[data > 127])}")
                    print("   Recomendação: Converter para int16 ou uint8.")
                else:
                    print("   ✅ Sem valores > 127 detectados.")
            else:
                print(f"✅ {path} usa o tipo de dado seguro: {dtype}")
    except Exception as e:
        print(f"❌ Erro ao ler {path}: {str(e)}")